In [1]:
using UncNLPTestSet
using DiffResults
using ForwardDiff: Dual, Partials, value, partials
using LinearAlgebra
nlp = SelectProgram("SROSENBR");

┌ Info: Precompiling UncNLPTestSet [ac41c6d5-581c-4fd8-9896-caf0766f302e]
└ @ Base loading.jl:1342
┌ Warning: TODO: Debug BDQRTIC
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BDQRTIC.jl:73
┌ Warning: TODO: Issue in CUTEst? See comment https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/broydn7d.jl#L50 
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BROYDN7D.jl:109
┌ Warning: TODO: Issue in CUTEst? See https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/brybnd.jl
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BRYBND.jl:110
┌ Warning: TODO: Debug CRAGGLVY
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/CRAGGLVY.jl:75
┌ Warning: TODO: Clean up DIXMAANB (and others) implementation. See comment block
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXMAANB.jl:108
┌ Warning: TODO: Implement DIXON3DQ
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXON3DQ.jl:1
┌ Warning: TOD

We illustrate overloading our objective function of the Seperable Rosenbrock problem with a Dual number, which returns a tuple containing two fields.
- objDual.value: the value of the objective function at the specified point (nlp.x0)
- objDual.partials: the derivative of the objective function in the specified direction (v1)

In [2]:
v1 = rand(nlp.n)
objDual = obj(nlp, Dual{1}.(nlp.x0, v1)) 
objDual.value ≈ obj(nlp, nlp.x0), objDual.partials[1] ≈ grad(nlp, nlp.x0)'v1

(true, true)

It behaves just like we wanted. We evaluated the objective function and the specified directional derivative in an economical fasion.

Now we extend this to the analytically specified gradient, to obtain curvature information of our objective function in a specified direction. (In our example, we choose a rand direction v1)


In [3]:
v1 = rand(nlp.n)
dual = Dual{2}.(nlp.x0, v1)
g_dual = similar(dual)
nlp.g!(g_dual, dual)

5000-element Vector{Dual{2, Float64, 1}}:
 Dual{2}(211.59999999999997,-67.74942322895767)
 Dual{2}(-87.99999999999999,38.090239918829774)
 Dual{2}(211.59999999999997,246.63867907462208)
 Dual{2}(-87.99999999999999,-77.08775560430045)
 Dual{2}(211.59999999999997,297.9407826229751)
 Dual{2}(-87.99999999999999,-98.89370713671946)
 Dual{2}(211.59999999999997,-433.5139939947571)
 Dual{2}(-87.99999999999999,181.62071436985482)
 Dual{2}(211.59999999999997,109.80785464118964)
 Dual{2}(-87.99999999999999,-30.015814959615984)
 Dual{2}(211.59999999999997,-56.170875405869715)
 Dual{2}(-87.99999999999999,37.20476264749931)
 Dual{2}(211.59999999999997,79.2059452863353)
           ⋮
 Dual{2}(211.59999999999997,732.0640959216609)
 Dual{2}(-87.99999999999999,-250.1364377523203)
 Dual{2}(211.59999999999997,239.83901620838463)
 Dual{2}(-87.99999999999999,-81.74825445079945)
 Dual{2}(211.59999999999997,-62.05852661016894)
 Dual{2}(-87.99999999999999,40.51650522577033)
 Dual{2}(211.59999999999997,0.0809787

In [4]:
function extract_partials!(p, dual, n) 
	for i in 1:n
		p[i, :] = dual[i].partials[:]
	end
	return p
end

extract_partials! (generic function with 1 method)

In [5]:
using LinearAlgebra
h = hessAD(nlp, nlp.x0);
p = zeros(nlp.n)
extract_partials!(p, g_dual, nlp.n);
h*v1 ≈ p

true

Thus, it worked in this instance as well. Their ought to be a better way of extracting the partials from the Dual. Now we build a preliminary `gAD` and the above test to a block of 10 directions.


In [19]:
function gAD(nlp, x::Vector{<:Real}, S::Matrix{<:Real})
    dual = Dual{1}.(x,  eachcol(S)...)
    result = Dual{1}.(zeros(nlp.n),  eachcol(S)...) # TODO: this is ridiculous
	nlp.g!(result, dual)
    g = similar(x)
    for i in 1:nlp.n
        S[i, :] = result[i].partials[:]
        g[i] = result[i].value
    end
    return g, S
end

gAD (generic function with 1 method)

In [20]:
S = rand(nlp.n, 10);
dual = gAD(nlp, nlp.x0, S)

([211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999  …  211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999, 211.59999999999997, -87.99999999999999], [682.32484895812 661.7034898707896 … 308.6036867290044 138.19364215961576; -242.82256207826464 -233.1344968107893 … -105.71248770383858 -40.94437696675605; … ; 553.4983515260164 -2.655990289504292 … -363.17695013230485 -125.81546902662981; -181.38644768205606 6.093763213556996 … 155.03258280533956 64.50264323844107])

We confirm that our dual contians the supplemental hessian information, that correspond to the directions of the columns of S. 

In [8]:
s1 = extract_partials!(similar(S), dual, nlp.n)

5000×10 Matrix{Float64}:
  -69.3352   486.32     412.788    …   588.923    338.69       45.5174
   31.2506  -156.496   -125.012       -202.994    -98.5139     -1.33103
  243.166    627.332    145.98         368.179    169.542     509.994
  -81.3048  -200.02     -29.9304      -110.604    -55.1672   -170.69
 1094.33    1175.68    1064.0         -308.961    521.745     783.323
 -388.486   -421.993   -372.423    …   130.016   -176.112    -270.467
   94.988    173.007    227.692        118.352     23.9639    976.765
  -26.7531   -36.9649   -78.0801       -19.4754    -6.57908  -350.558
  148.101    389.887    -26.5267       137.368   1188.16      537.005
  -49.2135  -139.084     20.4594       -39.313   -421.573    -189.101
  513.17     516.893     10.5078   …  1242.15     676.032     122.147
 -178.032   -181.485      1.13253     -444.831   -218.631     -34.9999
  470.278    686.766    640.669        259.464    729.149      37.2278
    ⋮                              ⋱                         

In [9]:
h = hessAD(nlp, nlp.x0)

5000×5000 Matrix{Float64}:
 1330.0  -480.0    -0.0    -0.0    -0.0  …    -0.0    -0.0    -0.0    -0.0
 -480.0   200.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
   -0.0    -0.0  1330.0  -480.0    -0.0       -0.0    -0.0    -0.0    -0.0
    0.0     0.0  -480.0   200.0     0.0        0.0     0.0     0.0     0.0
   -0.0    -0.0    -0.0    -0.0  1330.0       -0.0    -0.0    -0.0    -0.0
    0.0     0.0     0.0     0.0  -480.0  …     0.0     0.0     0.0     0.0
   -0.0    -0.0    -0.0    -0.0    -0.0       -0.0    -0.0    -0.0    -0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
   -0.0    -0.0    -0.0    -0.0    -0.0       -0.0    -0.0    -0.0    -0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
   -0.0    -0.0    -0.0    -0.0    -0.0  …    -0.0    -0.0    -0.0    -0.0
    0.0     0.0     0.0     0.0     0.0        0.0     0.0     0.0     0.0
   -0.0    -0.0    -0.0    -0.0    -0.0       -0.0    -0.0    -0.0    -0.

In [1]:
println( h*S[:, 1] ≈ s1[:, 1] )
println( h*S[:, 2] ≈ s1[:, 2] )
println( h*S[:, 3] ≈ s1[:, 3] )
println( h*S[:, 4] ≈ s1[:, 4] )
println( h*S[:, 5] ≈ s1[:, 5] )
println( h*S[:, 6] ≈ s1[:, 6] )
println( h*S[:, 7] ≈ s1[:, 7] )
println( h*S[:, 8] ≈ s1[:, 8] )
println( h*S[:, 9] ≈ s1[:, 9] )
println( h*S[:, 10] ≈ s1[:, 10] )

LoadError: UndefVarError: S not defined

In [1]:
M = rand(nlp.n, 8)

LoadError: UndefVarError: nlp not defined